# Molecular docking
***

**Note:** This notebook is part of the lecture series Molecular Modeling in Drug Design which is part of the Master of Drug Sciences curriculum. It is based on the talktorial created by the [Volkamer lab](https://volkamerlab.org/) at the Charité in Berlin ([Original Tutorial](https://projects.volkamerlab.org/teachopencadd/talktorials/T015_protein_ligand_docking.html)).


Authors:

- Manuel Sellner, 2023
- Peter Rüthemann, peter.ruethemann@unibas.ch, 2023
- Markus Lill, 2023

## Aim of this notebook
In this notebook, we will use molecular docking to predict the binding mode of a small molecule in a protein binding site. The *Androgen receptor ([AR](https://www.uniprot.org/uniprot/P10275))* will serve as a model system to explain important steps of a molecular docking workflow with the docking software [Smina](https://sourceforge.net/projects/smina/), a fork of Autodock Vina. The Androgen receptor is an interesting target because it is subject to high degrees of induced fit and different compounds (agonists and antagonists) may have completely different binding modes.

### References
- Molecular docking:
    - Pagadala _et al._, [_Biophy Rev_ (2017), __9__, 91-102](https://doi.org/10.1007/s12551-016-0247-1)
    - Meng _et al._, [_Curr Comput Aided Drug Des_ (2011), __7__, 2, 146-157](https://doi.org/10.2174/157340911795677602)
    - Gromski _et al._, [_Nat Rev Chem_ (2019), __3__, 119-128](https://doi.org/10.1038/s41570-018-0066-y)
- Docking and scoring function assessment:
    - Warren _et al._, [_J Med Chem_ (2006), __49__, 20, 5912-31](https://doi.org/10.1021/jm050362n)
    - Wang _et al._, [_Phys Chem Chem Phys_ (2016), __18__, 18, 12964-75](https://doi.org/10.1039/c6cp01555g)
    - Koes _et al._, [_J Chem Inf Model_ (2013), __53__, 8, 1893-1904](https://doi.org/10.1021/ci300604z)
    - Kimber _et al._, [_Int J Mol Sci_, (2021), __22__, 9, 1-34](https://doi.org/10.3390/ijms22094435)
    - McNutt _et al._, [_J Cheminform_ (2021), __13__, 43, 13-43](https://doi.org/10.1186/s13321-021-00522-2)
- Visual inspection of docking results: Fischer et al., [_J Med Chem_ (2021), __64__, 5, 2489–2500](https://doi.org/10.1021/acs.jmedchem.0c02227)
- Tools used
    - [OpenBabel](http://openbabel.org/wiki/Main_Page)
    - [Smina](https://sourceforge.net/projects/smina/)
    - [PyMol](https://pymol.org/2/)

## Import libraries and define locations

In [6]:
import io
from pathlib import Path
from os import path, makedirs
import subprocess

from rdkit import Chem
from rdkit.Chem import AllChem

from openbabel import pybel

extracted_ligand_path =  path.join('input', "2AMB_ligand.pdb")  # Co-crystallized ligand
protein_path = path.join('input', "2AMB_receptor.pdb")          # Protein without ligand (receptor)
output_dir = 'output'                                           # Define output directory

# Create the output directory if it does not exist yet
prepared_ligs_dir = path.join(output_dir, 'prepared_ligands')   # Directory for prepared ligands
docked_dir = path.join(output_dir, 'docked_ligands')            # Directory for docked ligands
log_dir = path.join(output_dir, 'log')

makedirs(log_dir, exist_ok=True)  
makedirs(docked_dir, exist_ok=True)  
makedirs(prepared_ligs_dir, exist_ok=True)  

Let's define the paths for all downstream analysis at the beginning. Before starting with Molecular Docking it always helpful to check the structures. In this exercise we will use [Pymol](https://pymol.org).

**Task 1** Visualize and open the receptor and ligand from the pdb structure 2AMB in Pymol.

## Prepare ligand

Next, we will take care of the ligand preparation. Since we want to re-dock the co-crystallized ligand into the binding site, we can retrieve the ligand in SMILES format directly from the Protein Data Bank (ligand id: [17H](https://www.rcsb.org/ligand/17H)). 

**NOTE:** We would be able to use the given ligand in `input/2AMB_ligand.pdb` to start our molecular docking approach. But since we know its conformation and exact position from the crystal structure, we would introduce a bias. To be sure we don't influence the algorithm we generate a totally new conformation from scratch. We therefore start from a SMILES string. That is a string representation of small molecules which doesn't represent any 3D information. From the SMILES string we generate one single conformation which fullfils chemical and physical properties.

### Generate conformation with forcefield

There are multiple ways how to generate chemical conformations. We will present you two options. The first method is slow because it uses a forcefield which iterativley optimizes the position of every atom and even adjusts the protonation state depending on a given pH. This method gives reasonable conformation of molecules but not necessarily represent a pose in the binding mode.

In [7]:
def generate_conformation_forcefield(smiles, pH=7.4, steps=10000):
    """
    Convert a SMILES string to a conformation in 3D iterativly optimized with a forcefield.

    Parameters
    ----------
    smiles: str
        SMILES string.
    pH: float
        pH for protonation.
    steps: int
        Deterimines number of iteration steps.
        
    Returns
    -------
    molecule: openbabel.pybel.Molecule
        The generated conformer of the molecule
    """
    
    # Read molecule from SMILES string
    molecule = pybel.readstring("smi", smiles)
    
    # add hydrogens at given pH
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    
    # generate 3D coordinates using a simple force field
    molecule.make3D(forcefield="mmff94s", steps=steps)
        
    return molecule

#### Compute the ligand conformation of the ligand 17H using OpenBabel

**Task 2**: Insert the SMILES string for the 17H ligand.

In [8]:
# Define the SMILES string of our molecule taken from protein databank
smi = 'CC[C@@]1(CC[C@@H]2[C@@]1(C=CC3=C4CCC(=O)C=C4CC[C@@H]23)CC)O'

# Generate the 3D structure
ligand_conformation = generate_conformation_forcefield(smi)

# Save a OpenBabel molecule as sdf file
ligand_babel = path.join(prepared_ligs_dir, 'ligand_babel.sdf')
ligand_conformation.write('sdf', ligand_babel, overwrite=True)

*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 1 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 5


### Generate conformation from a database

As a second conformation generation we use the experimental-torsion knowledge distance geometry method. [ETKDG](https://pubs.acs.org/doi/full/10.1021/acs.jcim.5b00654) combines distance geometries with experimental torsion angle preferences to quickly generate conformers of small molecules.

In [9]:
def generate_conformation_data(smiles, sanitize=True):
    """
    Convert a SMILES string to a conformation in 3D using a data-driven approach.

    Parameters
    ----------
    smiles: str
        SMILES string.
    sanitize: bool
        Sanitize the molecule by fixing small errors.
        
    Returns
    -------
    molecule: Chem.rdchem.Mol
        The generated conformer of the molecule
    """
    
    # Create a RDKit molecule from SMILES
    molecule = Chem.MolFromSmiles(smiles, sanitize=sanitize)   
    molecule = Chem.AddHs(molecule)

    # Initiate the ETKD parameters of the third version
    params = AllChem.ETKDGv3()
    params.useSmallRingTorsions = True

    # Generate a single conformation with the given parameters   
    AllChem.EmbedMultipleConfs(molecule, numConfs = 1 , params = params)
    return molecule

def save_sdf(m, target_path):
    """
    Saves a RD-Kit molecule as SDF file.

    Parameters
    ----------
    m: Chem.rdchem.Mol 
        The molecule to save
    target_path: str or pathlib.Path
        Location where file is saved
    """
    with Chem.SDWriter(target_path) as writer:
        writer.write(m)

#### Compute the ligand conformation using RDKit

In [10]:
# Generate the 3D structure using RDKit
ligand_conformation_RDKit = generate_conformation_data(smi)

# Save a RDKit molecule as sdf file
output_file_rdkit = path.join(prepared_ligs_dir, 'ligand-RDKit.sdf')
save_sdf(ligand_conformation_RDKit, output_file_rdkit)

The newly generated conformation is similar but not identical to the extracted ligand conformation. 

**Question 1:** What are advantages and disadvantages of the two methods? In which scenarios would you prefer one method over the other?

**Task 3** Open both generated 3D structures of the ligand in PyMol and compare it visually. Are they identical?

## Binding site definition

In theory, it would be possible to perform a docking calculation considering the whole protein as binding site. However, due to the known limitations of docking algorithms, it is always recommended to restrict the binding site to a certain region of interest to increase performance and accuracy. Here, we can use the coordinates of the co-crystallized ligand to define the binding site with a rectangular box. 

* We will calculate the geometric center of the molecule, which represents the center of the rectangular box,
* as well as the length of the molecule in each dimension, which corresponds to the edge lengths of the rectangular box. 
* Also, we will add a 5 Å buffer in each dimension to allow the correct placements of ligands that are bigger than the co-crystallized ligands or bind in a different fashion.


**Task 4:** Complete the function calculate_geometric_center which computes the center of the binding site and 
    the dimension of the binding site. 
    Note: Replace the underlines with the missing code. Multiple solutions are possible.

    HINT: Try to solve the problem first in a step-by-step manner on pen and paper.

In [11]:
def calculate_geometric_center(ligand, buffer=5):
    """
    Calculate the geometric center of a ligand and 
    the size the binding site with a given buffer 
    around the ligand.

    Parameters
    ----------
    ligand: Chem.rdchem.Mol
        The co-crystallized ligand
    buffer: int
        The buffer distance to add around the ligand. (Angstrom)
        
    Returns
    -------
    pocket_center: numpy.ndarray
        The center coordinates of the ligand
    pocket_size = numpy.ndarray
        The size of the binding pocket
    """
    # Get the coordinates of all ligand atoms
    lig_coordinates = ligand.GetConformer().GetPositions()
    
    # Calculate the xyz coordinates of the center of the ligand
    pocket_center = lig_coordinates.sum(axis=0) / lig_coordinates.shape[0]
    
    # Calculate the dimension (size) of the ligand and add additional 5 Angstrom to every dimension
    pocket_size = lig_coordinates.max(axis=0) - lig_coordinates.min(axis=0) + buffer
    
    return (pocket_center, pocket_size)

### Compute the geometric center

In [12]:
# Import the ligand found in the pdb crystal structure
extracted_ligand = Chem.MolFromPDBFile(extracted_ligand_path, removeHs=False, sanitize=True)

# Calculate the the geometric center and the size (pocket size) of the binding site
pocket_center, pocket_size = calculate_geometric_center(extracted_ligand)

print(f"pseudoatom {pocket_center[0]} {pocket_center[1]} {pocket_center[2]}")

pseudoatom 27.519352941176475 1.6504117647058825 4.352235294117646


## Docking calculation: Redocking

Finally, we can proceed to the actual docking calculation. Smina is a command line tool, which currently does not provide a Python API. 

* However, we can write a function that will take care of generating the proper command and execute the calculation. 
* Besides providing the ligand, protein and pocket information, we can also alter the behaviour of the docking calculation by setting the number of docking poses to be generated and by changing the exhaustiveness, which controls the number of independent docking calculations.

**NOTE:** Depending on the CPU speed and the docking settings this step may take a few minutes.

In [13]:
def run_smina(ligand, protein, output, pocket_center, pocket_size, log, num_poses=10, exhaustiveness=8):
    """
    Perform docking with Smina.

    Parameters
    ----------
    ligand: str or pathlib.Path
        Path to ligand file that should be docked. Formats: mol2, sdf, pdbqt, pdb
    protein: str or pathlib.Path
        Path to protein file that should be docked to. Formats: pdb, pdbqt
    output: str or pathlib.Path
        Path to which docking poses should be saved. Formats: sdf, mol2, pdb
    pocket_center: iterable of float or int
        Coordinates defining the center of the binding site.
    pocket_size: iterable of float or int
        Lengths of edges defining the binding site.
    log: path
        Location where smina log is saved.
    num_poses: int
        Maximum number of poses to generate.
    exhaustiveness: int
        Accuracy of docking calculations.
             
    Instead of pocket_center and pocket_size, the autobox
    function could be used.
    """

    # Define the terminal command in a list. (Required from the Python subprocess module)
    command =  ["smina",
                "--ligand",   ligand,
                "--receptor", protein,
                "--out",      output,
                "--center_x", str(pocket_center[0]),
                "--center_y", str(pocket_center[1]),
                "--center_z", str(pocket_center[2]),
                "--size_x",   str(pocket_size[0]),
                "--size_y",   str(pocket_size[1]),
                "--size_z",   str(pocket_size[2]),
                "--num_modes", str(num_poses),
                "--exhaustiveness", str(exhaustiveness)
               ]

    # Execute command in terminal over a Python subprocess
    log_text = subprocess.check_output(command, universal_newlines=True)

    # Write Log to the log folder
    with open(log, "w") as f:
        f.writelines(log_text)

### Execute the molecular docking process

In [14]:
# Define the output file that should be generated
docked_molecule = path.join(docked_dir, 'docked_poses.sdf')
log_file = path.join(log_dir, 'smina_2AMB_17H.txt')

# Run smina
smina_infos = run_smina(ligand=output_file_rdkit,
                        protein=protein_path,
                        output=docked_molecule,
                        pocket_center=pocket_center,
                        pocket_size=pocket_size,
                        log=log_file
                        )

The output is a file containing the docking poses and a text based description of the docking results including an estimated affinity and the root mean square deviation (RMSD) of each pose from the pose with the highest predicted affinity ([Vina output description](http://vina.scripps.edu/manual.html#output)).

### Docking results visualization

Now, let us analyze the docking results. Were we able to reproduce the binding mode observed in the experimentally resolved structure?

First, let us display the complex with the original (co-crystallized) pose. In order to display contacts between the protein and ligand, non-covalent interactions can be visualized in PyMol.

**Task 5** Let's also check if the output files were generated. Open the docked ligand in PyMol. We will perform this task in the plenum.

**Question 2:** Which kind of non-bonding interactions does the ligand form?

**Task 6:** Visually compare the docked ligand pose to the co-crystallized ligand?

### Calculate RMSD

Since we want to quantify the quality of the generated pose and not just perform a visual inspection (Task 6), we calculate the root mean squared deviation (RMSD) between the docked and co-crystallized poses.

**NOTE:** Since smina removes non-polar hydrogen atoms, we need to remove them in the co-crystallized pose as well. This can be done by setting `removeHs=True`.

In [15]:
# Load the co-crystallized pose
reference = Chem.MolFromPDBFile(extracted_ligand_path, removeHs=True, sanitize=True)

# Load the docked pose
docked = Chem.MolFromMolFile(docked_molecule, removeHs=False, sanitize=True, strictParsing=False)

# Calculate the RMSD
rmsd = AllChem.CalcRMS(reference, docked)

print(f'RMSD between reference and docked ligand: {rmsd:.2f} Å')

RMSD between reference and docked ligand: 0.73 Å


Evidently, smina is able to re-dock this ligand very nicely. A RMSD value below 1 A is considered to be good. However, re-docking is usually one of the easiest tasks when it comes to docking.

## Cross-docking

Let us see if we can dock other ligands to the same receptor. Docking ligands co-crystallized in one crystal structure to another crystal structured is called cross-docking. We will use the following ligands. All the ligands in the following ligand are supposed to bind to the same receptor.

| PDB ID   |      Name         |  SMILES |
|:---------|:------------------|:--------|
| 1T65     |  DHT              | C\[C@\]12CCC(=O)C\[C@@H\]1CC\[C@@H\]3\[C@@H\]2CC\[C@\]4(\[C@H\]3CC\[C@@H\]4O)C |
| 1Z95     |  Bicalutamide     | C\[C@\](CS(=O)(=O)c1ccc(cc1)F)(C(=O)Nc2ccc(c(c2)C(F)(F)F)C#N)O |
| 2AX6     |  Hydroxyflutamide | CC(C)(C(=O)Nc1ccc(c(c1)C(F)(F)F)\[N+\](=O)\[O-\])O |
| 2AX7     |  S-1              | C\[C@\](COc1ccc(cc1)F)(C(=O)Nc2ccc(c(c2)C(F)(F)F)\[N+\](=O)\[O-\])O |
| 2AX9     |  R-3              | C\[C@\](CBr)(C(=O)Nc1ccc(c(c1)C(F)(F)F)\[N+\](=O)\[O-\])O |
| 2PNU     |  EM-5744          | C\[C@\]12CCC(=O)C\[C@@H\]1CC\[C@@H\]3\[C@@H\]2CC\[C@\]4(\[C@H\]3CC\[C@@H\]4O)CCOCc5cc(cc(c5)F)F |

### Generate the 3D structure of Hydroxyfultamide
Here, we will start by docking Hydroxyflutamide, a nonsteroidal antiandrogen.

In [16]:
hflutamide_smiles = "CC(C)(C(=O)Nc1ccc(c(c1)C(F)(F)F)[N+](=O)[O-])O"
hflutamide_conformation = generate_conformation_data(hflutamide_smiles)

# Save SDF file
output_file_hflutamide = path.join(prepared_ligs_dir, 'hflutamide.sdf')
save_sdf(hflutamide_conformation,output_file_hflutamide)

### Dock the ligand to the same receptor we have used before.

In [17]:
hflutamide_docked = path.join(docked_dir, 'hflutamide.sdf')
smina_infos = run_smina(ligand=output_file_hflutamide,
                        protein=protein_path,
                        output=hflutamide_docked,
                        pocket_center=pocket_center,
                        pocket_size=pocket_size,
                        log=path.join(log_dir, 'hflutamide.txt')
                        )

Let's again quantify the quality of the pose by calculating the RMSD to the reference pose. You can find the reference poses for all cross-docking ligands in the direcory `input/cross-docking`.

In [18]:
# Load the docked poses
reference = Chem.MolFromMolFile(path.join("input", "cross-docking", "2AX6_hydroxyflutamide_reference.sdf"), removeHs=True)
docked_molecules = Chem.SDMolSupplier(hflutamide_docked, sanitize=True)

# Calculate and print the RMSDs for all generated poses
print("--- RMSDs for docked poses ---")
for i, pose in enumerate(docked_molecules, 1):
    rmsd = AllChem.CalcRMS(pose, reference)
    print(f"Pose {i}: {rmsd:.2f}")

--- RMSDs for docked poses ---
Pose 1: 1.15
Pose 2: 2.88
Pose 3: 3.02
Pose 4: 6.44
Pose 5: 3.02
Pose 6: 6.21
Pose 7: 6.24
Pose 8: 6.62
Pose 9: 7.17
Pose 10: 6.19


### Crossdocking Exercise and Discussion

Using a combination of four free software tools ([OpenBabel](http://openbabel.org/wiki/Main_Page), [RDKit](https://www.rdkit.org/), [Smina](https://sourceforge.net/projects/smina/), PyMol), we were able to perform docking calculations and visualize the results.  
We started with a comparatively easy task: The reproduction of an experimentally observed binding mode. In this case, the binding pocket has the ideal protein conformations to harbor the ligand. Docking of other ligands can be more complicated, since they may prefer binding to a different protein conformation. This could be observed in the cross-docking task. Considering certain side chains as flexible may improve docking results in this situation, but will fail if the protein backbone is changing its conformation upon ligand binding. Another common practice is so called ensemble docking in which multiple ligands are docked to an ensemble of different crystal structures from the same protein to maximize the chance of finding a fitting protein conformation for each ligand.

**Task 7:** Repeat the process for the remaining ligands in the table. We provided the template for the crossdocking example of all ligands. Complete the missing parts.

In [19]:
ligands = { 'DHT': 'C[C@]12CCC(=O)C[C@@H]1CC[C@@H]3[C@@H]2CC[C@]4([C@H]3CC[C@@H]4O)C',
            'Bicalutamide': 'C[C@](CS(=O)(=O)c1ccc(cc1)F)(C(=O)Nc2ccc(c(c2)C(F)(F)F)C#N)O',
            'Hydroxyflutamide': 'CC(C)(C(=O)Nc1ccc(c(c1)C(F)(F)F)[N+](=O)[O-])O',        
            'R-3': 'C[C@](CBr)(C(=O)Nc1ccc(c(c1)C(F)(F)F)[N+](=O)[O-])O',
            'EM-5744': 'C[C@]12CCC(=O)C[C@@H]1CC[C@@H]3[C@@H]2CC[C@]4([C@H]3CC[C@@H]4O)CCOCc5cc(cc(c5)F)F',
            'S-1': 'C[C@](COc1ccc(cc1)F)(C(=O)Nc2ccc(c(c2)C(F)(F)F)[N+](=O)[O-])O',
            '2AMB': 'CC[C@@]1(CC[C@@H]2[C@@]1(C=CC3=C4CCC(=O)C=C4CC[C@@H]23)CC)O'
          }

ligands_references = {'DHT': '1T65_DHT_reference.sdf',
                      'Bicalutamide': '1Z95_bicalutamide_reference.sdf',
                      'Hydroxyflutamide': '2AX6_hydroxyflutamide_reference.sdf',
                      'R-3': '2AX9_R-3_reference.sdf',
                      'EM-5744': '2PNU_EM-5744_reference.sdf',
                      'S-1': '2AX7_S-1_reference.sdf',
                      '2AMB': '2AMB_ligand.sdf'
                     }

In [20]:
# Iterate over all ligands and dock against the same prepared crystal structure
for ligand_name, smile in ligands.items():
    
    # Generate conformation of ligand
    conformation = generate_conformation_data(smile) # generate_conformation_forcefield(smile) #
    prepared_ligand = path.join(prepared_ligs_dir, f'{ligand_name}.sdf')
    
    # Save as sdf file
    save_sdf(conformation, prepared_ligand)
  
    # Docking
    docked_ligand = path.join(docked_dir, f'{ligand_name}.sdf')
    
    smina_infos = run_smina(ligand=prepared_ligand,
                            protein=protein_path,
                            output=docked_ligand,
                            pocket_center=pocket_center,
                            pocket_size=pocket_size,
                            log = path.join(log_dir, f'{ligand_name}.txt')
                           )
    # Load the docked poses   
    docked_molecules = Chem.SDMolSupplier(docked_ligand, sanitize=True)

    # Calculate and print the RMSDs for all generated poses
    reference = Chem.MolFromMolFile(path.join("input", "cross-docking", ligands_references[ligand_name]), removeHs=True)
    
    print(f"\n--- RMSDs for docked poses with ligand {ligand_name} ---")
    for i, pose in enumerate(docked_molecules, 1):
        rmsd = AllChem.CalcRMS(pose, reference)
        print(f"Pose {i}: {rmsd:.2f}")


--- RMSDs for docked poses with ligand DHT ---
Pose 1: 6.49
Pose 2: 6.17
Pose 3: 1.47
Pose 4: 1.65
Pose 5: 2.70
Pose 6: 2.84
Pose 7: 6.55

--- RMSDs for docked poses with ligand Bicalutamide ---
Pose 1: 4.00
Pose 2: 4.58
Pose 3: 5.06
Pose 4: 5.34
Pose 5: 4.96
Pose 6: 3.30
Pose 7: 4.34
Pose 8: 3.55

--- RMSDs for docked poses with ligand Hydroxyflutamide ---
Pose 1: 1.16
Pose 2: 2.87
Pose 3: 6.46
Pose 4: 3.08
Pose 5: 6.23
Pose 6: 1.17
Pose 7: 6.28
Pose 8: 2.95
Pose 9: 7.08
Pose 10: 1.79

--- RMSDs for docked poses with ligand R-3 ---
Pose 1: 3.02
Pose 2: 1.56
Pose 3: 3.19
Pose 4: 2.01
Pose 5: 0.86
Pose 6: 3.20
Pose 7: 6.68
Pose 8: 7.33
Pose 9: 6.78
Pose 10: 6.69

--- RMSDs for docked poses with ligand EM-5744 ---
Pose 1: 6.12
Pose 2: 5.79
Pose 3: 3.84

--- RMSDs for docked poses with ligand S-1 ---
Pose 1: 3.75
Pose 2: 4.17
Pose 3: 6.69
Pose 4: 6.97
Pose 5: 2.95
Pose 6: 4.51
Pose 7: 4.97
Pose 8: 4.14
Pose 9: 2.75
Pose 10: 5.24

--- RMSDs for docked poses with ligand 2AMB ---
Pose 1: 0.

**Question 3** What do you observe and how can these observations be explained? What are potential biophysical explanantions for this behaviour?